# Membership Inference over Diffusion-models-based Synthetic Tabular Data (MIDST) Challenge @ SaTML 2025.

## White Box Single Table Competition
Welcome to the MIDST challenge!

The MIDST challenge is a multi-track competition aiming to quantitatively evaluate the privacy of synthetic tabular data generated by diffusion models, with a specific focus on its resistance to membership inference attacks (MIAs).

This competition focuses on White Box MIA on tabular diffusion models trained on a single table transaction dataset. The schema of the transaction dataset is as follows:
| trans_id | account_id | trans_date | trans_type | operation | amount  | balance  | k_symbol | bank | account |
|----------|------------|------------|------------|-----------|---------|----------|----------|------|---------|
| integer  | integer    | integer    | integer    | integer   | float   | float    | integer  | integer | integer |

MIA will be performed over two state-of-the-art methods [TabSyn](https://arxiv.org/pdf/2310.09656) and [TabDDPM](https://arxiv.org/pdf/2209.15421). A collection of TabSyn and TabDDPM models will be trained on random subsets of the transaction dataset. The goal is to create an approach (MIA) that can distinguish between samples used to train a model (train data) and other data randomly sampled from the transaction dataset (holdout data) given the model and it's output synthetic data. The `final` set includes 20 models, each with its own set of challenge points (ie train and holdout data), to evaluate solutions on. To facilitate designing an attack, 30 `train` models are provided with comprehensive information about the model, training data and output synthetic data. Additionally, 20 `dev` models are provided to assist in evaluating the effectiveness of attacks prior to making a final submission to the `final` set. Participants can choose to perform MIA over one of or both TabSyn and TabDDPM. In the case of both, the attack that obtains the highest score will be used to rank the submission. A high level summary of the competition is below:
![wbox_diagram_final](https://github.com/user-attachments/assets/2ebb5eed-a6e3-433a-8769-4310b7fbc822)

This notebook will walk you through the process of creating and packaging a submission to the white box single table challenge.

## Package Imports and Evironment Setup

Ensure that you have installed the proper dependenices to run the notebook. The environment installation instructions are available [here](https://github.com/VectorInstitute/MIDSTModels/tree/main/starter_kits). Now that we have verfied we have the proper packages installed, lets import them and define global variables:

In [2]:
import csv
import os
import random
import zipfile

from pathlib import Path
from functools import partial
from typing import Callable, Any

import numpy as np
import torch

from tqdm.notebook import tqdm
from data import get_challenge_points, get_challenge_labels
from metrics import get_tpr_at_fpr



In [3]:
%cd ..

c:\Users\ksush\attacks\MIDSTModels


C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
from midst_models.single_table_TabDDPM.complex_pipeline import tabddpm_whitebox_load_pretrained

In [5]:
TABDDPM_DATA_DIR = "tabddpm_white_box"
TABSYN_DATA_DIR = "tabsyn_white_box"

In [6]:

def get_FLAGS():
    def FLAGS(x): return x
    FLAGS.T = 5000
    FLAGS.ch = 128
    FLAGS.ch_mult = [1, 2, 2, 2]
    FLAGS.attn = [1]
    FLAGS.num_res_blocks = 2
    FLAGS.dropout = 0.1
    FLAGS.beta_1 = 0.001
    FLAGS.beta_T = 0.2

    return FLAGS

FLAGS = get_FLAGS()


In [15]:
import torch
import numpy as np
import logging
import csv
import os
from typing import Dict, Type
import seaborn as sns
from torch.nn.functional import normalize
import midst_models.attack.components as components
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd

class EpsGetter(components.EpsGetter):
    def __call__(self, xt: torch.Tensor, condition: torch.Tensor = None, noise_level=None, t: int = None) -> torch.Tensor:
        # Access the diffusion model from the dictionary structure
        model = self.model[(None, 'trans')]['diffusion']

        t = torch.ones([xt.shape[0]], device=xt.device).long() * t
        return model._denoise_fn(xt, timesteps=t)

ATTACKERS: Dict[str, Type[components.DDIMAttacker]] = {
    "PIA": components.PIA,
    "PIAN": components.PIAN,
}


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


def attack(base_dir, attacker_name="PIA", attack_num=30, interval=10, seed=0):
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    # logger.addHandler(RichHandler())

    logger.info("loading the attacked model...")

    # Initialize attacker
    phases = ["train"]
    # phases = ["dev", "final"]

    logger.info("attack start...")
    for phase in phases:
        root = os.path.join(base_dir, phase)
        for model_folder in sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])):
            path = os.path.join(root, model_folder)

            model = tabddpm_whitebox_load_pretrained(path)
            attacker = ATTACKERS[attacker_name](
                torch.from_numpy(np.linspace(FLAGS.beta_1, FLAGS.beta_T, FLAGS.T)).to(DEVICE), interval, attack_num, EpsGetter(model), lp=1)

            # Move the points to GPU. Shape is [num_sample (200), num_features (8)]
            challenge_points = get_challenge_points(path)

            # DF with shape [200, 1] containing binary label whether the point is in the training set or not
            challenge_labels = get_challenge_labels(path)

            # Get raw predictions. torch.Size is [num_sample (200), num_attackers] 
            raw_predictions = torch.stack([attacker(cp.to(DEVICE).float()) for cp in challenge_points])

            # Mean here, assuming the num_attackers stabilized the distance
            raw_predictions_means = raw_predictions.mean(dim=1).cpu().detach().numpy()
            
            # Convert DataFrame to numpy array
            challenge_labels_np = challenge_labels.values.squeeze()

            # Filter arrays before plotting
            non_member_distances = raw_predictions_means[challenge_labels_np == 0]
            member_distances = raw_predictions_means[challenge_labels_np == 1]

            # Create a figure with two subplots side by side
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
            
            # Plot 1: Enhanced histogram
            bins = np.linspace(min(raw_predictions_means), max(raw_predictions_means), 50)
            
            ax1.hist(non_member_distances, bins=bins, alpha=0.7, 
                    label='Non-member', color='blue', density=True)
            ax1.hist(member_distances, bins=bins, alpha=0.7,
                    label='Member', color='red', density=True)
            
            # Add KDE curves to better show the distribution
            sns.kdeplot(data=non_member_distances, ax=ax1, color='darkblue', 
                    linewidth=2, label='Non-member KDE')
            sns.kdeplot(data=member_distances, ax=ax1, color='darkred',
                    linewidth=2, label='Member KDE')
            
            ax1.set_xlabel('Distance')
            ax1.set_ylabel('Density')
            ax1.set_title('Distribution of Distances\nwith Kernel Density Estimation')
            ax1.legend()
            ax1.grid(True, alpha=0.3)

            # Plot 2: Box plot to show statistical distribution
            df_plot = pd.DataFrame({
                'Distance': raw_predictions_means,
                'Type': ['Member' if label == 1 else 'Non-member' for label in challenge_labels_np]
            })
            
            sns.boxplot(x='Type', y='Distance', data=df_plot, ax=ax2,
                    palette={'Member': 'red', 'Non-member': 'blue'})
            
            # Add individual points with jitter
            sns.stripplot(x='Type', y='Distance', data=df_plot, ax=ax2,
                        alpha=0.3, size=4, jitter=0.2,
                        palette={'Member': 'darkred', 'Non-member': 'darkblue'})
            
            ax2.set_title('Statistical Distribution of Distances')
            ax2.grid(True, alpha=0.3)
            
            # Adjust layout and save
            plt.tight_layout()
            plt.savefig(os.path.join(path, f"new_distance_distribution_T={interval}.png"), dpi=300, bbox_inches='tight')
            plt.close()

            # Add statistical summary to the data
            stats_summary = pd.DataFrame({
                'Metric': ['Mean', 'Median', 'Std Dev', '25th Percentile', '75th Percentile'],
                'Member': [
                    member_distances.mean(),
                    np.median(member_distances),
                    member_distances.std(),
                    np.percentile(member_distances, 25),
                    np.percentile(member_distances, 75)
                ],
                'Non-member': [
                    non_member_distances.mean(),
                    np.median(non_member_distances),
                    non_member_distances.std(),
                    np.percentile(non_member_distances, 25),
                    np.percentile(non_member_distances, 75)
                ]
            })
            
            # Save both raw data and statistics
            distance_data = pd.DataFrame({
                'Distance': raw_predictions_means,
                'Is_Member': challenge_labels_np
            })
            distance_data.to_csv(os.path.join(path, "distance_data.csv"), index=False)
            stats_summary.to_csv(os.path.join(path, "distance_statistics.csv"), index=False)

            # Convert to binary predictions based on membership
            normalized_preds = []
            for pred_batch in raw_predictions:
                binary_preds = (pred_batch <= 1000000).float()
                normalized_preds.append(binary_preds)

            final_predictions = torch.stack(normalized_preds)
            predictions_cpu = final_predictions.cpu().detach().numpy()

            with open(os.path.join(path, "prediction.csv"), mode="w", newline="") as file:
                writer = csv.writer(file)
                for value in predictions_cpu.squeeze():
                    writer.writerow([value])


In [16]:
attack(base_dir="tabddpm_white_box",
        attacker_name="PIA",
        attack_num=5,
        interval=200)

Checkpoint found, loading...
distance tensor([1.2766e+08, 1.8029e+08, 2.2829e+08, 2.7203e+08, 3.1280e+08],
       grad_fn=<SumBackward1>)
distance tensor([184512.4375, 259775.7344, 326731.8750, 388231.8125, 443458.7812],
       grad_fn=<SumBackward1>)
distance tensor([155293.0156, 226478.4688, 287526.7812, 343427.2500, 395041.6875],
       grad_fn=<SumBackward1>)
distance tensor([2.9737e+08, 4.1956e+08, 5.3064e+08, 6.3231e+08, 7.2682e+08],
       grad_fn=<SumBackward1>)
distance tensor([122896.5391, 178362.8438, 226865.6406, 271408.8438, 312368.9375],
       grad_fn=<SumBackward1>)
distance tensor([1.4634e+08, 2.0666e+08, 2.6167e+08, 3.1181e+08, 3.5853e+08],
       grad_fn=<SumBackward1>)
distance tensor([ 61207.4531,  86738.7188, 107328.8438, 125620.9531, 142188.6719],
       grad_fn=<SumBackward1>)
distance tensor([6.6719e+07, 9.3660e+07, 1.1775e+08, 1.4030e+08, 1.6100e+08],
       grad_fn=<SumBackward1>)


C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator QuantileTransformer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


distance tensor([151476.6250, 212540.4531, 267617.4062, 318256.3438, 363341.5000],
       grad_fn=<SumBackward1>)
distance tensor([263629.2188, 370401.6250, 469569.5625, 558279.6250, 636462.0000],
       grad_fn=<SumBackward1>)
distance tensor([109317.6797, 158294.6094, 200336.8594, 238527.3125, 274023.6875],
       grad_fn=<SumBackward1>)
distance tensor([130636.1875, 188750.2812, 239201.1875, 285078.0000, 327665.0938],
       grad_fn=<SumBackward1>)
distance tensor([191005.1875, 279337.9062, 356539.6875, 426171.9375, 490778.0312],
       grad_fn=<SumBackward1>)
distance tensor([125080.8750, 181408.2656, 230346.4375, 274671.8125, 315555.5312],
       grad_fn=<SumBackward1>)
distance tensor([184823.0625, 261931.8125, 331814.9688, 395166.4062, 451285.5312],
       grad_fn=<SumBackward1>)
distance tensor([267916.6562, 376866.4375, 473234.0938, 561656.1250, 640500.3750],
       grad_fn=<SumBackward1>)
distance tensor([176521.2812, 253101.9688, 318850.3750, 378892.2188, 435646.6875],
     

C:\Users\ksush\AppData\Local\Temp\ipykernel_65572\3337890583.py:100: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='Type', y='Distance', data=df_plot, ax=ax2,
C:\Users\ksush\AppData\Local\Temp\ipykernel_65572\3337890583.py:104: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.stripplot(x='Type', y='Distance', data=df_plot, ax=ax2,


Checkpoint found, loading...
distance tensor([158650.4844, 227881.7188, 287816.8750, 343191.1250, 394977.0625],
       grad_fn=<SumBackward1>)
distance tensor([205861.4062, 296970.0938, 376033.5000, 448360.1250, 515932.4062],
       grad_fn=<SumBackward1>)
distance tensor([147942.2188, 212775.4375, 268961.9688, 320628.1250, 369393.1562],
       grad_fn=<SumBackward1>)
distance tensor([190687.0156, 265570.0312, 330349.3750, 392936.3750, 452242.1250],
       grad_fn=<SumBackward1>)
distance tensor([236541.0781, 341250.1562, 432589.6875, 516046.2188, 593734.6875],
       grad_fn=<SumBackward1>)
distance tensor([160053.8438, 229587.3438, 289042.5938, 343899.5312, 394820.9375],
       grad_fn=<SumBackward1>)
distance tensor([165335.2344, 238533.2812, 301798.9062, 359472.4375, 413555.3438],
       grad_fn=<SumBackward1>)


...

C:\Users\ksush\AppData\Local\Temp\ipykernel_65572\3337890583.py:100: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='Type', y='Distance', data=df_plot, ax=ax2,
C:\Users\ksush\AppData\Local\Temp\ipykernel_65572\3337890583.py:104: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.stripplot(x='Type', y='Distance', data=df_plot, ax=ax2,


Checkpoint found, loading...
distance tensor([2.8606e+08, 4.3807e+08, 5.4489e+08, 6.3909e+08, 7.1710e+08],
       grad_fn=<SumBackward1>)
distance tensor([333089.0938, 479662.6250, 607017.9375, 724863.8750, 836621.6875],
       grad_fn=<SumBackward1>)
distance tensor([ 668643.5000,  974139.0000, 1242384.3750, 1485171.1250, 1711116.8750],
       grad_fn=<SumBackward1>)
distance tensor([1495360.5000, 2161109.0000, 2733830.0000, 3265269.5000, 3766279.7500],
       grad_fn=<SumBackward1>)
distance tensor([ 865506.8125, 1256491.8750, 1593788.5000, 1919713.7500, 2232360.5000],
       grad_fn=<SumBackward1>)
distance tensor([ 406392.1250,  592911.3750,  758425.4375,  914985.6250, 1061640.3750],
       grad_fn=<SumBackward1>)
distance tensor([18045982., 28985716., 36880332., 43099960., 48778540.],
       grad_fn=<SumBackward1>)
distance tensor([1.8150e+08, 2.7818e+08, 3.4606e+08, 4.0594e+08, 4.5555e+08],
       grad_fn=<SumBackward1>)


C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator QuantileTransformer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


distance tensor([1301034.1250, 1864591.8750, 2368181.2500, 2828542.5000, 3264390.0000],
       grad_fn=<SumBackward1>)
distance tensor([1.9375e+08, 2.9712e+08, 3.6963e+08, 4.3363e+08, 4.8666e+08],
       grad_fn=<SumBackward1>)
distance tensor([387560.5625, 560690.5000, 711729.3125, 847863.5000, 976970.0000],
       grad_fn=<SumBackward1>)
distance tensor([ 887730.5625, 1276616.1250, 1617087.3750, 1936114.3750, 2240063.7500],
       grad_fn=<SumBackward1>)
distance tensor([1151800.2500, 1671977.0000, 2131020.7500, 2557841.5000, 2961659.7500],
       grad_fn=<SumBackward1>)
distance tensor([1778322.3750, 2557979.5000, 3243564.0000, 3874922.0000, 4481209.0000],
       grad_fn=<SumBackward1>)
distance tensor([1.4370e+08, 2.2034e+08, 2.7414e+08, 3.2160e+08, 3.6092e+08],
       grad_fn=<SumBackward1>)
distance tensor([ 967516.1250, 1389076.6250, 1760175.2500, 2110417.7500, 2440718.7500],
       grad_fn=<SumBackward1>)
distance tensor([ 887677.0000, 1277804.5000, 1612115.5000, 1922735.0000, 

C:\Users\ksush\AppData\Local\Temp\ipykernel_65572\3337890583.py:100: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='Type', y='Distance', data=df_plot, ax=ax2,
C:\Users\ksush\AppData\Local\Temp\ipykernel_65572\3337890583.py:104: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.stripplot(x='Type', y='Distance', data=df_plot, ax=ax2,


## Scoring

Let's see how the attack does on `train`, for which we have the ground truth.
When preparing a submission, you can use part of `train` to develop an attack and a held-out part to evaluate your attack.

In [8]:
import numpy as np
from sklearn.metrics import roc_auc_score
import os
from typing import List, Tuple

def safe_load_data(filepath: str, is_prediction: bool = True) -> np.ndarray:
    """
    Safely load data from CSV files with error handling and debugging.
    
    Args:
        filepath: Path to the CSV file
        is_prediction: Whether this is a prediction file (True) or label file (False)
    
    Returns:
        numpy.ndarray: Loaded and validated data
    """
    try:
        if is_prediction:
            # Read the file line by line and parse each array
            predictions = []
            with open(filepath, 'r') as f:
                for line in f:
                    # Remove brackets and split by spaces
                    clean_line = line.strip().replace('[', '').replace(']', '')
                    # Convert space-separated strings to floats
                    row = np.array([float(x) for x in clean_line.split()])
                    predictions.append(row)
            return np.array(predictions)
        else:
            # For label files, skip header and use numpy
            return np.loadtxt(filepath, skiprows=1)
            
    except Exception as e:
        print(f"Error loading {filepath}: {str(e)}")
        print(f"File contents (first few lines):")
        with open(filepath, 'r') as f:
            print(f.read(500))
        raise

def get_tpr_at_fpr(y_true: np.ndarray, y_score: np.ndarray, target_fpr: float = 0.1) -> float:
    """Calculate TPR at a specific FPR threshold."""
    # Input validation
    if not isinstance(y_true, np.ndarray) or not isinstance(y_score, np.ndarray):
        raise TypeError("Inputs must be numpy arrays")
    if y_true.shape != y_score.shape:
        raise ValueError(f"Shape mismatch: y_true {y_true.shape} != y_score {y_score.shape}")
    
    # Sort scores and corresponding truth values
    desc_score_indices = np.argsort(y_score, kind="mergesort")[::-1]
    y_score = y_score[desc_score_indices]
    y_true = y_true[desc_score_indices]
    
    n_neg = np.sum(y_true == 0)
    n_pos = np.sum(y_true == 1)
    
    # Handle edge cases
    if n_neg == 0 or n_pos == 0:
        print("Warning: Found no positive or negative samples")
        return 0.0
        
    tpr = np.cumsum(y_true) / n_pos
    fpr = np.cumsum(1 - y_true) / n_neg
    
    for i in range(len(fpr)):
        if fpr[i] >= target_fpr:
            return tpr[i]
    return 1.0

def evaluate_membership_inference(base_dirs: List[str]) -> Tuple[float, float]:
    """
    Evaluate membership inference attack results across multiple directories.
    
    Args:
        base_dirs: List of base directories containing prediction files
        
    Returns:
        tuple: (best_tpr_at_fpr, best_auc)
    """
    tpr_at_fpr_list = []
    auc_list = []
    
    for base_dir in base_dirs:
        predictions = []
        solutions = []
        root = os.path.join(base_dir, "train")
        
        if not os.path.exists(root):
            print(f"Warning: Directory not found: {root}")
            continue
            
        model_folders = sorted(os.listdir(root), key=lambda d: int(d.split('_')[1]))
        if not model_folders:
            print(f"Warning: No model folders found in {root}")
            continue
        
        # Load and process all predictions and solutions
        for model_folder in model_folders:
            path = os.path.join(root, model_folder)
            pred_path = os.path.join(path, "prediction.csv")
            label_path = os.path.join(path, "challenge_label.csv")
            
            if not (os.path.exists(pred_path) and os.path.exists(label_path)):
                print(f"Warning: Missing files in {path}")
                continue
            
            try:
                # Load predictions
                pred = safe_load_data(pred_path, is_prediction=True)
                predictions.append(pred)
                
                # Load ground truth
                solution = safe_load_data(label_path, is_prediction=False)
                if solution.ndim == 0:
                    solution = solution.reshape(1)
                solutions.append(solution)
                
            except Exception as e:
                print(f"Error processing folder {model_folder}: {str(e)}")
                continue
        
        if not predictions or not solutions:
            print(f"Warning: No valid data found in {base_dir}")
            continue
            
        try:
            # Concatenate all predictions and solutions
            predictions = np.concatenate(predictions)
            solutions = np.concatenate(solutions)
            
            print(f"\nData shapes for {os.path.basename(base_dir)}:")
            print(f"Predictions shape: {predictions.shape}")
            print(f"Solutions shape: {solutions.shape}")
            
            # Calculate metrics for each attacker's predictions
            num_attackers = predictions.shape[1]
            for attacker_idx in range(num_attackers):
                attacker_preds = predictions[:, attacker_idx]
                
                # Basic data validation
                if np.any(np.isnan(attacker_preds)):
                    print(f"Warning: NaN values found in predictions for attacker {attacker_idx}")
                    continue
                    
                # Calculate metrics
                tpr_at_fpr = get_tpr_at_fpr(solutions, attacker_preds)
                tpr_at_fpr_list.append(tpr_at_fpr)
                
                try:
                    auc = roc_auc_score(solutions, attacker_preds)
                    auc_list.append(auc)
                except ValueError as e:
                    print(f"Warning: Could not calculate AUC for attacker {attacker_idx}: {e}")
                    continue
                
                print(f"{os.path.basename(base_dir)} Attacker {attacker_idx + 1}:")
                print(f"  TPR at FPR==10%: {tpr_at_fpr:.4f}")
                print(f"  AUC: {auc:.4f}")
                
        except Exception as e:
            print(f"Error processing directory {base_dir}: {str(e)}")
            continue
    
    # Get best scores
    final_tpr_at_fpr = max(tpr_at_fpr_list) if tpr_at_fpr_list else 0.0
    final_auc = max(auc_list) if auc_list else 0.0
    
    print(f"\nBest scores across all attackers:")
    print(f"Final Train Attack TPR at FPR==10%: {final_tpr_at_fpr:.4f}")
    print(f"Final Train Attack AUC: {final_auc:.4f}")
    
    return final_tpr_at_fpr, final_auc

In [ ]:
base_dirs = [TABDDPM_DATA_DIR]
final_tpr, final_auc = evaluate_membership_inference(base_dirs)

## Packaging the submission

Now we can store the predictions into a zip file, which you can submit to CodaBench. Importantly, we create a single zip file for dev and final. The structure of the submission is as follows:

```
└── root_folder
    ├── tabsyn_white_box
    │   ├── dev
    │   │   └── tabsyn_#
    │   │       └── prediction.csv
    │   └── final
    │       └── tabsyn_#
    │           └── prediction.csv
    └── tabddpm_white_box
        ├── dev 
        │   └── tabddpm_#
        │       └── prediction.csv
        └── final 
            └── tabddpm_# 
                └── prediction.csv
```
**Note:** The `root_folder` can have any name but it is important all of the subdirectories follow the above structure and naming conventions. 

If a participant is looking to submit an attack for only one of TabSyn and TabDDPM, they can simply omit the other directory (ie `tabddpm_white_box` or `tabsyn_white_box` from the root_folder).

In [10]:
import os
import zipfile
import pandas as pd
import numpy as np
import re  # For fixing malformed lists

def parse_numpy_array(array_str):
    """Fix NumPy-style array formatting and convert it to a Python list."""
    array_str = array_str.strip()  # Remove leading/trailing spaces
    array_str = re.sub(r'\s+', ',', array_str)  # Replace spaces with commas
    array_str = array_str.replace("[,", "[").replace(",]", "]")  # Fix misplaced commas
    return eval(array_str)  # Convert string to list

with zipfile.ZipFile("white_box_single_table_submission.zip", 'w') as zipf:
    for phase in ["dev", "final"]:
        for base_dir in [TABDDPM_DATA_DIR]:
            root = os.path.join(base_dir, phase)
            for model_folder in sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])):
                path = os.path.join(root, model_folder)
                if not os.path.isdir(path): 
                    continue

                file = os.path.join(path, "prediction.csv")
                if os.path.exists(file):
                    # Load CSV
                    df = pd.read_csv(file, header=None)

                    # Convert NumPy-style arrays to proper lists
                    df = df[0].apply(lambda x: parse_numpy_array(x) if isinstance(x, str) else x)

                    # Compute mean for each row
                    df_mean = df.apply(lambda x: np.mean(x) if isinstance(x, list) else x)

                    # Save the new CSV
                    df_mean.to_csv(file, index=False, header=False)

                    # Add to ZIP
                    arcname = os.path.relpath(file, os.path.dirname(base_dir))
                    zipf.write(file, arcname=arcname)
                else:
                    raise FileNotFoundError(f"`prediction.csv` not found in {path}.")


The generated white_box_single_table_submission.zip can be directly submitted to the dev phase in the CodaBench UI. Although this submission contains your predictions for both the dev and final set, you will only receive feedback on your predictions for the dev phase. The predictions for the final phase will be evaluated once the competiton ends using the most recent submission to the dev phase.